In [1]:
# common python modules
import sys
import math
#import warnings
#warnings.filterwarnings("ignore")


# EDA and data visualization modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('data_files/fifa21_male2.csv')

/Users/sebastianvier/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Reusable functions

def slice_by_values(arr, init, end):
    init = list(arr).index(init)
    end = list(arr).index(end)
    return list(arr[init:end + 1])

In [4]:
for column in data.columns:
    print(column)

ID
Name
Age
OVA
Nationality
Club
BOV
BP
Position
Player Photo
Club Logo
Flag Photo
POT
Team & Contract
Height
Weight
foot
Growth
Joined
Loan Date End
Value
Wage
Release Clause
Contract
Attacking
Crossing
Finishing
Heading Accuracy
Short Passing
Volleys
Skill
Dribbling
Curve
FK Accuracy
Long Passing
Ball Control
Movement
Acceleration
Sprint Speed
Agility
Reactions
Balance
Power
Shot Power
Jumping
Stamina
Strength
Long Shots
Mentality
Aggression
Interceptions
Positioning
Vision
Penalties
Composure
Defending
Marking
Standing Tackle
Sliding Tackle
Goalkeeping
GK Diving
GK Handling
GK Kicking
GK Positioning
GK Reflexes
Total Stats
Base Stats
W/F
SM
A/W
D/W
IR
PAC
SHO
PAS
DRI
DEF
PHY
Hits
LS
ST
RS
LW
LF
CF
RF
RW
LAM
CAM
RAM
LM
LCM
CM
RCM
RM
LWB
LDM
CDM
RDM
RWB
LB
LCB
CB
RCB
RB
GK
Gender


In [10]:
# How many teams have 20 or more players?
print('Number of teams with more than 20 players:')
print(len(data.iloc[:,5].value_counts()[data.iloc[:,5].value_counts() >= 20]))

# Number of total teams
print('\nTotal number of teams')
print(data.iloc[:,5].nunique())

Number of teams with more than 20 players:
446

Total number of teams
917


### Decide the best replacement for a transfered player in Sevilla FC

 First we are going to decide what player is going to be transfered.
 We select a player who is leaving on 2021 and has the highest wage.
A player with a high wage is likely a player who is playing most matches.

##### But first we need to convert the column wage.

In [40]:
sevilla_fc = data[data['Club'] == 'Sevilla FC']
sorted_index = sevilla_fc['Wage'].map(lambda x : int(x[1:-1])).sort_values(ascending=False).index
## This codes strip the K and € sing gets the index to sort the players
sevilla_fc.loc[sorted_index[:10]][['Name', 'Nationality', 'Contract', 'Wage', 'Age']]
## That index is used then on the sevilla_fc to sort the playes by wage.
## The [:10] is used to only get the first 10 players.

,Name,Nationality,Contract,Wage,Age
569,Jesús Navas,Spain,2017 ~ 2021,€43K,34
7631,M. Acuña,Argentina,2020 ~ 2024,€41K,28
4593,L. Ocampos,Argentina,2019 ~ 2024,€40K,25
6702,Diego Carlos,Brazil,2019 ~ 2024,€39K,27
4048,Suso,Spain,2020 ~ 2025,€39K,26
1061,I. Rakitić,Croatia,2020 ~ 2024,€39K,32
1925,Fernando,Brazil,2019 ~ 2022,€39K,32
2482,L. de Jong,Netherlands,2019 ~ 2023,€38K,29
2845,Escudero,Spain,2015 ~ 2021,€31K,30
7624,Joan Jordán,Spain,2019 ~ 2023,€30K,25


So the player that is leaving is **Sergio Escudero**. Actually he is not playing in Sevilla anymore.

##### How do you decide what is the best replacement for a player?
- How do you decide the amount you could pay?
- How do you determine wether a player is good or not for a giving position?
- Is there a way to determine wether or not a player suits the team they are going?
- How can we determine wether a player price is cheap or expensive?


#### What are the options?

 ###### First we check the players that have the same best position and the best position of Escudero

In [48]:
bp_escudero = data.iloc[2845]['BP']
position_escudero = data.iloc[2845]['Position']
escudero_wage = int(data.iloc[2845]['Wage'][1:-1])

## Amount of players that have exactly the same position

## This is to reduce the data I am currently using.
slice_by_values(data.columns, 'ID', 'Age')
info_data = data[slice_by_values(data.columns, 'ID', 'Contract')]

# Same position and same best position
condition_1 = info_data['Position'] == position_escudero
condition_2 = info_data['BP'] == bp_escudero

# Around the same wage condition_3
def filter_condition_3(value, threshold=.10):
    if value[-1] != 'K':
        return False
    if int(value[1:-1]) > (escudero_wage + escudero_wage*threshold):
        return False
    if int(value[1:-1]) < (escudero_wage - escudero_wage*threshold):
        return False
    else:
        return True
    
condition_3 = info_data['Wage'].apply(lambda x : filter_condition_3(x, .35)) 

## Create a global condition
conditions = condition_1 & condition_2 & condition_3
lb_players = info_data[conditions]

In [49]:
lb_players[['Name', 'Club','Nationality','Wage']]

,Name,Club,Nationality,Wage
2072,M. Rybus,Lokomotiv Moscow,Poland,€35K
2705,F. N'Sakala,Beşiktaş JK,DR Congo,€38K
2845,Escudero,Sevilla FC,Spain,€31K
4420,Toño García,Levante UD,Spain,€21K
4882,Lucas Lima,Al Ahli,Brazil,€33K
5850,Kim Jin Su,Al Nassr,Korea Republic,€22K
6010,J. Mojica,Atalanta,Colombia,€40K
7631,M. Acuña,Sevilla FC,Argentina,€41K
8872,Ronaldo Esler,Atlético Mineiro,Brazil,€27K
11781,M. Olivera,Getafe CF,Uruguay,€22K


In [50]:
lb_players_complete = data[condition_1 & condition_2]

In [54]:
lb_players_complete['Value']
lb_players_complete['Wage']

555      €13K
672        €0
1001     €20K
1187     €51K
1640      €2K
         ... 
16489     €2K
16702     €3K
16836     €3K
16875    €500
17030    €500
Name: Wage, Length: 131, dtype: object

In [65]:
# sns.scatterplot(x='Wage', y='Value', data='lb_players_complete')

def euros2numbers(value):
    pass


def remove_sign(val):
    if (type(val) == str) and (val[0] == '€'):
        return val[1:]
    else:
        return val

lb_players_complete['Wage'].apply(lambda x : remove_sign(x))


555      13K
672        0
1001     20K
1187     51K
1640      2K
        ... 
16489     2K
16702     3K
16836     3K
16875    500
17030    500
Name: Wage, Length: 131, dtype: object